<a href="https://colab.research.google.com/github/samvardhan777/financial_banking_app_sentimental_analysis/blob/main/fine_tune_llama2_banking_review.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/BFSI/final_bfsi_lablel.csv')

In [ ]:
df

,Financial_context,annotation_id,annotator,content,created_at,id,language,lead_time,score,sentiment,thumbsUpCount,updated_at
0,Financial,14,1,1] Customer service is really nice.,2024-03-31T06:58:47.318794Z,1332,en,9.348,3,Positive,0,2024-03-31T06:58:47.318813Z
1,Non Financial,15,1,2] It is not very easy to set up mobile bankin...,2024-03-31T06:59:03.859506Z,1333,en,12.568,3,Neutral,0,2024-03-31T06:59:03.859527Z
2,Non Financial,16,1,I understand it's for safety but still a hassle.,2024-03-31T06:59:46.583294Z,1334,en,21.842,3,Neutral,0,2024-03-31T06:59:46.583307Z
3,Non Financial,17,1,3] App itself is easy to use.,2024-03-31T06:59:58.593876Z,1335,en,7.949,3,Positive,0,2024-03-31T06:59:58.593889Z
4,Non Financial,18,1,"4] Unfortunately, you can't view your card num...",2024-03-31T07:00:26.848093Z,1336,en,16.955,3,Negative,0,2024-03-31T07:00:26.848109Z
...,...,...,...,...,...,...,...,...,...,...,...,...
383,Non Financial,382,1,I don't know what's is the problem?,2024-03-31T11:39:35.667314Z,2532,en,9.276,4,Negative,0,2024-03-31T11:39:35.667327Z
384,Financial,383,1,It's very disappointing.,2024-03-31T11:39:43.659762Z,2533,en,6.336,4,Negative,0,2024-03-31T11:39:43.659780Z
385,Non Financial,370,1,2nd largest bank in India but mobile app is no...,2024-03-31T11:29:04.279001Z,2534,en,4.080,4,Negative,0,2024-03-31T11:29:04.279018Z
386,Financial,384,1,What a pity.,2024-03-31T11:40:03.224609Z,2535,en,25.356,4,Negative,0,2024-03-31T11:40:03.224623Z


In [3]:
final_df = df[['Financial_context','sentiment','content']]

In [4]:
final_df

,Financial_context,sentiment,content
0,Financial,Positive,1] Customer service is really nice.
1,Non Financial,Neutral,2] It is not very easy to set up mobile bankin...
2,Non Financial,Neutral,I understand it's for safety but still a hassle.
3,Non Financial,Positive,3] App itself is easy to use.
4,Non Financial,Negative,"4] Unfortunately, you can't view your card num..."
...,...,...,...
383,Non Financial,Negative,I don't know what's is the problem?
384,Financial,Negative,It's very disappointing.
385,Non Financial,Negative,2nd largest bank in India but mobile app is no...
386,Financial,Negative,What a pity.


In [144]:
from sklearn.model_selection import train_test_split
import pandas as pd

# Set aside a portion of the data for evaluation
eval_fraction = 0.1  # For example, 10% of the data for evaluation
train_test_data, X_eval = train_test_split(final_df, test_size=eval_fraction, random_state=42)

X_train = []
X_test = []

for sentiment in ["Positive", "Neutral", "Negative"]:
    subset = train_test_data[train_test_data['sentiment'] == sentiment]

    # Check the size of the subset
    subset_size = len(subset)

    # Set train and test size dynamically based on the subset size
    train_size = min(300, int(subset_size * 0.7))  # For example, 70% for training
    test_size = min(100, subset_size - train_size)  # Remaining for testing, but not more than 100

    # Perform the split
    if subset_size > 0:  # Proceed only if the subset is not empty
        train, test = train_test_split(subset,
                                       train_size=train_size,
                                       test_size=test_size,
                                       random_state=42)
        X_train.append(train)
        X_test.append(test)

# Concatenate the lists to form the full training and testing sets
if X_train and X_test:
    X_train = pd.concat(X_train).sample(frac=1, random_state=10).reset_index(drop=True)
    X_test = pd.concat(X_test).reset_index(drop=True)
    # For evaluation, you might want to ensure it's not overlapping with train/test
else:
    print("No data available for training and testing sets.")

remaining_idx = [idx for idx in df.index if idx not in X_train.index and idx not in X_test.index]
X_eval = df.loc[remaining_idx]

# If you want to sample from X_eval for balanced sentiment representation
X_eval = (X_eval
          .groupby('sentiment', group_keys=False)
          .apply(lambda x: x.sample(min(len(x), 50), random_state=10, replace=True)))
X_eval = X_eval.reset_index(drop=True)

In [172]:
def generate_prompt(data_point):
    return f"""
            Analyze the sentiment of the statement enclosed in square brackets,
            determine if it is positive, neutral, or negative, and return the answer as
            the corresponding sentiment label "Positive" or "Neutral" or "Negative"

           [{data_point['content']}] = {data_point['sentiment']}
            """.strip() + EOS_TOKEN

In [173]:
def generate_test_prompt(data_point):
    return f"""
           Analyze the sentiment of the statement enclosed in square brackets,
            determine if it is positive, neutral, or negative, and return the answer as
            the corresponding sentiment label "Positive" or "Neutral" or "Negative"

        [{data_point['content']}] =

    """.strip()

In [145]:
X_train['prompt'] = X_train.apply(generate_prompt, axis=1)
X_test['prompt'] = X_test.apply(generate_test_prompt, axis=1)
X_eval['prompt'] = X_eval.apply(generate_prompt, axis=1)

In [146]:
y_true = X_test[['sentiment', 'Financial_context']]
y_true_sentiment = X_test['sentiment']
y_true_financial = X_test['Financial_context']

In [147]:
pd.DataFrame(X_train.apply(generate_prompt, axis=1), columns=["prompt"])

,prompt
0,generate_prompt\n Analyze the senti...
1,generate_prompt\n Analyze the senti...
2,generate_prompt\n Analyze the senti...
3,generate_prompt\n Analyze the senti...
4,generate_prompt\n Analyze the senti...
...,...
238,generate_prompt\n Analyze the senti...
239,generate_prompt\n Analyze the senti...
240,generate_prompt\n Analyze the senti...
241,generate_prompt\n Analyze the senti...


In [13]:
!pip install -q -U datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.1 MB/s eta 0:00:00


In [14]:
from datasets import Dataset

train_data = Dataset.from_pandas(X_train[['prompt']])
eval_data = Dataset.from_pandas(X_eval[['prompt']])
test_data = Dataset.from_pandas(X_test[['prompt']])

In [174]:
from sklearn.metrics import (accuracy_score,
                             classification_report,
                             confusion_matrix)
import numpy as np
def evaluate(y_true, y_pred):
    labels = ['Positive', 'Neutral', 'Negative']
    mapping = {'Positive': 2, 'Neutral': 1,'none':1, 'Negative': 0}
    def map_func(x):
        return mapping.get(x, 1)

    y_true = np.vectorize(map_func)(y_true)
    y_pred = np.vectorize(map_func)(y_pred)

    # Calculate accuracy
    accuracy = accuracy_score(y_true=y_true, y_pred=y_pred)
    print(f'Accuracy: {accuracy:.3f}')

    # Generate accuracy report
    unique_labels = set(y_true)  # Get unique labels

    for label in unique_labels:
        label_indices = [i for i in range(len(y_true))
                         if y_true[i] == label]
        label_y_true = [y_true[i] for i in label_indices]
        label_y_pred = [y_pred[i] for i in label_indices]
        accuracy = accuracy_score(label_y_true, label_y_pred)
        print(f'Accuracy for label {label}: {accuracy:.3f}')

    # Generate classification report
    class_report = classification_report(y_true=y_true, y_pred=y_pred)
    print('\nClassification Report:')
    print(class_report)

    # Generate confusion matrix
    conf_matrix = confusion_matrix(y_true=y_true, y_pred=y_pred, labels=[0, 1, 2])
    print('\nConfusion Matrix:')
    print(conf_matrix)

In [179]:
import os
del os.environ['LC_ALL']

In [180]:
%%capture
import torch
major_version, minor_version = torch.cuda.get_device_capability()
# Must install separately since Colab has torch 2.2.1, which breaks packages
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
if major_version >= 8:
    # Use this for new GPUs like Ampere, Hopper GPUs (RTX 30xx, RTX 40xx, A100, H100, L40)
    !pip install --no-deps packaging ninja einops flash-attn xformers trl peft accelerate bitsandbytes
else:
    # Use this for older GPUs (V100, Tesla T4, RTX 20xx)
    !pip install --no-deps xformers trl peft accelerate bitsandbytes
pass

NotImplementedError: A UTF-8 locale is required. Got ANSI_X3.4-1968

In [102]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
    "unsloth/llama-2-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",
    "unsloth/gemma-7b-it-bnb-4bit", # Instruct version of Gemma 7b
    "unsloth/gemma-2b-bnb-4bit",
    "unsloth/gemma-2b-it-bnb-4bit", # Instruct version of Gemma 2b
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/mistral-7b-bnb-4bit", # Choose ANY! eg teknium/OpenHermes-2.5-Mistral-7B
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

EOS_TOKEN = tokenizer.eos_token

==((====))==  Unsloth: Fast Mistral patching release 2024.3
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.25.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


In [19]:
import bitsandbytes as bnb
def find_all_linear_names(model):
  cls = bnb.nn.Linear4bit #if args.bits == 4 else (bnb.nn.Linear8bitLt if args.bits == 8 else torch.nn.Linear)
  lora_module_names = set()
  for name, module in model.named_modules():
    if isinstance(module, cls):
      names = name.split('.')
      lora_module_names.add(names[0] if len(names) == 1 else names[-1])
    if 'lm_head' in lora_module_names: # needed for 16-bit
      lora_module_names.remove('lm_head')
  return list(lora_module_names)

In [20]:
modules = find_all_linear_names(model)
print(modules)

['k_proj', 'down_proj', 'gate_proj', 'up_proj', 'v_proj', 'o_proj', 'q_proj']


In [169]:
from tqdm import tqdm
def predict(X_test, model, tokenizer):
    y_pred_sentiment = []
    y_pred_financial = []
    for i in tqdm(range(len(X_test))):
        prompt = X_test.iloc[i]["prompt"]
        input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")
        outputs = model.generate(**input_ids, pad_token_id=tokenizer.eos_token_id, max_new_tokens=1, temperature=0.0)
        result = tokenizer.decode(outputs[0])
        answer = result.split("=")[-1].lower().strip()

        sentiment = "none"  # Default sentiment
        if "positive" in answer.lower():
            sentiment = "Positive"
        elif "negative" in answer.lower():
            sentiment = "Negative"
        elif "neutral" in answer.lower():
            sentiment = "Neutral"

        y_pred_sentiment.append(sentiment)

        print("----------Result------------------")
        print(result)
        print("----------Answer------------------")
        print(answer)
        print("-----------Sentiment------------------")
        print(sentiment)


    return  y_pred_sentiment

In [170]:
 y_pred_sentiment =predict(X_test, model, tokenizer)

  2%|▏         | 2/106 [00:00<00:12,  8.60it/s]

----------Result------------------
<bos>Analyze the sentiment of the news headline enclosed in square brackets, 
            determine if it is positive, neutral, or negative, and return the answer as 
            the corresponding sentiment label "positive" or "neutral" or "negative"

            [Good] = positive
----------Answer------------------
positive
-----------Sentiment------------------
Positive
----------Result------------------
<bos>Analyze the sentiment of the news headline enclosed in square brackets, 
            determine if it is positive, neutral, or negative, and return the answer as 
            the corresponding sentiment label "positive" or "neutral" or "negative"

            [Great app!] = positive
----------Answer------------------
positive
-----------Sentiment------------------
Positive


  5%|▍         | 5/106 [00:00<00:10, 10.09it/s]

----------Result------------------
<bos>Analyze the sentiment of the news headline enclosed in square brackets, 
            determine if it is positive, neutral, or negative, and return the answer as 
            the corresponding sentiment label "positive" or "neutral" or "negative"

            [Best bank] = "
----------Answer------------------
"
-----------Sentiment------------------
none
----------Result------------------
<bos>Analyze the sentiment of the news headline enclosed in square brackets, 
            determine if it is positive, neutral, or negative, and return the answer as 
            the corresponding sentiment label "positive" or "neutral" or "negative"

            [Loans taken on AC/ Credit card also to be shown.] = negative
----------Answer------------------
negative
-----------Sentiment------------------
Negative
----------Result------------------
<bos>Analyze the sentiment of the news headline enclosed in square brackets, 
            determine if it is positiv

  7%|▋         | 7/106 [00:00<00:09, 10.38it/s]

...

----------Result------------------
<bos>Analyze the sentiment of the news headline enclosed in square brackets, 
            determine if it is positive, neutral, or negative, and return the answer as 
            the corresponding sentiment label "positive" or "neutral" or "negative"

            [Blocker as we are not able to access the application] = "
----------Answer------------------
"
-----------Sentiment------------------
none
----------Result------------------
<bos>Analyze the sentiment of the news headline enclosed in square brackets, 
            determine if it is positive, neutral, or negative, and return the answer as 
            the corresponding sentiment label "positive" or "neutral" or "negative"

            [Very bad experience even rm not picking call] = "
----------Answer------------------
"
-----------Sentiment------------------
none
----------Result------------------
<bos>Analyze the sentiment of the news headline enclosed in square brackets, 
            deter

 97%|█████████▋| 103/106 [00:10<00:00, 10.78it/s]

----------Result------------------
<bos>Analyze the sentiment of the news headline enclosed in square brackets, 
            determine if it is positive, neutral, or negative, and return the answer as 
            the corresponding sentiment label "positive" or "neutral" or "negative"

            [SMS VerificationFailed: App is not able to send messages to bind with sim card from samsung secure folder.] = negative
----------Answer------------------
negative
-----------Sentiment------------------
Negative
----------Result------------------
<bos>Analyze the sentiment of the news headline enclosed in square brackets, 
            determine if it is positive, neutral, or negative, and return the answer as 
            the corresponding sentiment label "positive" or "neutral" or "negative"

            [It's sending message from my registered number but never verified before the timeout and just couple of seconds after timeout, it gets received.] = "
----------Answer------------------
"
--

100%|██████████| 106/106 [00:10<00:00,  9.94it/s]

----------Result------------------
<bos>Analyze the sentiment of the news headline enclosed in square brackets, 
            determine if it is positive, neutral, or negative, and return the answer as 
            the corresponding sentiment label "positive" or "neutral" or "negative"

            [My transfer I cant make has been explained away as maybe need to clear cache but no one can explain how to do that.] = negative
----------Answer------------------
negative
-----------Sentiment------------------
Negative
----------Result------------------
<bos>Analyze the sentiment of the news headline enclosed in square brackets, 
            determine if it is positive, neutral, or negative, and return the answer as 
            the corresponding sentiment label "positive" or "neutral" or "negative"

            [I giving one star because there is no more option to select less than one star.] = negative
----------Answer------------------
negative
-----------Sentiment------------------
Negat

In [171]:
 y_pred_sentiment

['Positive',
 'Positive',
 'none',
 'Negative',
 'none',
 'none',
 'Positive',
 'Positive',
 'Positive',
 'none',
 'none',
 'none',
 'none',
 'none',
 'none',
 'none',
 'none',
 'none',
 'none',
 'none',
 'none',
 'none',
 'none',
 'none',
 'Positive',
 'Positive',
 'none',
 'Positive',
 'none',
 'none',
 'Positive',
 'none',
 'none',
 'none',
 'none',
 'Positive',
 'none',
 'none',
 'none',
 'Positive',
 'none',
 'none',
 'none',
 'none',
 'Positive',
 'none',
 'none',
 'none',
 'Positive',
 'none',
 'none',
 'Positive',
 'none',
 'Negative',
 'none',
 'none',
 'none',
 'none',
 'Negative',
 'none',
 'Positive',
 'none',
 'Negative',
 'none',
 'none',
 'none',
 'none',
 'none',
 'none',
 'Negative',
 'Negative',
 'none',
 'Negative',
 'Negative',
 'none',
 'none',
 'none',
 'none',
 'none',
 'Negative',
 'none',
 'none',
 'none',
 'none',
 'Negative',
 'Negative',
 'none',
 'none',
 'none',
 'none',
 'none',
 'Negative',
 'none',
 'Negative',
 'none',
 'none',
 'none',
 'Negative',
 '

In [117]:
from transformers import (AutoModelForCausalLM,
                          AutoTokenizer,
                          BitsAndBytesConfig,
                          TrainingArguments,
                          pipeline,
                          logging)

compute_dtype = getattr(torch, "float16")

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
)


model = AutoModelForCausalLM.from_pretrained(
   'google/gemma-2b',
    device_map="auto",
    quantization_config=bnb_config,
)

model.config.use_cache = False
model.config.pretraining_tp = 1

max_seq_length = 2048
tokenizer = AutoTokenizer.from_pretrained('google/gemma-2b', max_seq_length=max_seq_length)
EOS_TOKEN = tokenizer.eos_token

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/555 [00:00<?, ?B/s]

In [142]:
def generate_prompt(data_point):
    return f"""generate_prompt
            Analyze the sentiment of the news headline enclosed in square brackets,
            determine if it is positive, neutral, or negative, and return the answer as
            the corresponding sentiment label "positive" or "neutral" or "negative"

            [{data_point["content"]}] = {data_point["sentiment"]}
            """.strip() + EOS_TOKEN

def generate_test_prompt(data_point):
    return f"""
            Analyze the sentiment of the news headline enclosed in square brackets,
            determine if it is positive, neutral, or negative, and return the answer as
            the corresponding sentiment label "positive" or "neutral" or "negative"

            [{data_point["content"]}] =

            """.strip()